计算YOLOV3的Anchor Box
===

In [5]:
import os
import sys
import numpy as np
from tqdm import tqdm

In [2]:
def iou(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    :param box: tuple or array, shifted to the origin (i. e. width and height)
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou_ = intersection / (box_area + cluster_area - intersection)

    return iou_

In [3]:
def avg_iou(boxes, clusters):
    """
    Calculates the average Intersection over Union (IoU) between a numpy array of boxes and k clusters.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: average IoU as a single float
    """
    return np.mean([np.max(iou(boxes[i], clusters)) for i in range(boxes.shape[0])])


def translate_boxes(boxes):
    """
    Translates all the boxes to the origin.
    :param boxes: numpy array of shape (r, 4)
    :return: numpy array of shape (r, 2)
    """
    new_boxes = boxes.copy()
    for row in range(new_boxes.shape[0]):
        new_boxes[row][2] = np.abs(new_boxes[row][2] - new_boxes[row][0])
        new_boxes[row][3] = np.abs(new_boxes[row][3] - new_boxes[row][1])
    return np.delete(new_boxes, [0, 1], axis=1)

In [4]:
def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param k: number of clusters
    :param dist: distance function
    :return: numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]

    while True:
        for row in range(rows):
            distances[row] = 1 - iou(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

# 2.计算VOC数据集的Anchor box

In [6]:
# 需要加载yolo训练数据和lable
def load_dataset(path):
    jpegimages = os.path.join(path, 'JPEGImages')
    if not os.path.exists(jpegimages):
        print('no JPEGImages folders, program abort')
        sys.exit(0)
    labels_txt = os.path.join(path, 'labels')
    if not os.path.exists(labels_txt):
        print('no labels folders, program abort')
        sys.exit(0)
 
    label_file = os.listdir(labels_txt)
    print('label count: {}'.format(len(label_file)))
    dataset = []
 
    for label in tqdm(label_file):
        with open(os.path.join(labels_txt, label), 'r') as f:
            txt_content = f.readlines()
 
        for line in txt_content:
            line_split = line.split(' ')
            roi_with = float(line_split[len(line_split)-2])
            roi_height = float(line_split[len(line_split)-1])
            if roi_with == 0 or roi_height == 0:
                continue
            dataset.append([roi_with, roi_height])
            # print([roi_with, roi_height])
 
    return np.array(dataset)

In [7]:
# 根文件夹
ROOT_PATH = '/Users/JJZHK/data/input/VOC/'
# 聚类的数目
CLUSTERS = 9
# 模型中图像的输入尺寸，默认是一样的
SIZE = 416

data = load_dataset(ROOT_PATH)
print("loading data done.")
out = kmeans(data, k=CLUSTERS)   #对训练样本聚类
# clusters = [[10,13],[16,30],[33,23],[30,61],[62,45],[59,119],[116,90],[156,198],[373,326]]
# out= np.array(clusters)/416.0

print(out)
print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(out[:, 0] * SIZE, out[:, 1] * SIZE))
 
ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))
#
# tensor([[ 3.6250,  2.8125],
#         [ 4.8750,  6.1875],
#         [11.6562, 10.1875]])
#
# 10,13,  16,30,  33,23,  30,61,    62,45,  59,119,  116,90,  156,198,  373,326
# 21,37   58,150  30,90   111,105   57,54   245,176  106,224  182,310   353,350

100%|██████████| 21503/21503 [00:01<00:00, 18776.71it/s]
loading data done.
[[0.074      0.20266667]
 [0.32       0.288     ]
 [0.152      0.14114114]
 [0.412      0.72533333]
 [0.65994236 0.46133333]
 [0.054      0.088     ]
 [0.136      0.336     ]
 [0.84224828 0.86133333]
 [0.232      0.536     ]]
Accuracy: 69.17%
Boxes:
 [ 30.784      133.12        63.232      171.392      274.53602305
  22.464       56.576      350.37528604  96.512     ]-[ 84.30933333 119.808       58.71471471 301.73866667 191.91466667
  36.608      139.776      358.31466667 222.976     ]
Ratios:
 [0.37, 0.4, 0.43, 0.57, 0.61, 0.98, 1.08, 1.11, 1.43]


# 3.计算COCO数据集的Anchor Box

In [ ]:
ROOT_PATH = '/Users/JJZHK/data/input/COCO/'
# 聚类的数目
CLUSTERS = 9
# 模型中图像的输入尺寸，默认是一样的
SIZE = 416

zip = zipfile.ZipFile(os.path.join(ROOT_PATH, "train2017.zip"))

annFile = os.path.join(ROOT_PATH, "annotations", "instances_train2017.json")
print(annFile)
coco = COCO(annFile)
class_ids = sorted(coco.getCatIds()) # all classes

lines = []
dataset = []
for id in class_ids:
    print("id:%d" % id)
    image_id_list = list(coco.getImgIds(catIds=[id]))
    for image_id in tqdm(image_id_list):
        info = coco.imgs[image_id]
        width = info["width"]
        height = info['height']
        annIds = coco.getAnnIds(imgIds=image_id, iscrowd=None)
        ann_info = coco.loadAnns(annIds)
        for ann in ann_info:
            bbox = ann["bbox"]
            roi_width = float(bbox[2] / width)
            roi_height = float(bbox[3] / height)
            if roi_width == 0 or roi_height == 0:
                continue
            dataset.append([roi_width, roi_height])

data = np.array(dataset)
print("loading data done.")
out = kmeans(data, k=CLUSTERS)  # 对训练样本聚类

print(out)
print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(out[:, 0] * SIZE, out[:, 1] * SIZE))

ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))